# 1. Input data

Download yeast reads:

SRR941816: fermentation 0 minutes replicate 1 

`ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941816/SRR941816.fastq.gz` (413 Mb)

SRR941817: fermentation 0 minutes replicate 2 

`ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz` (455 Mb)

SRR941818: fermentation 30 minutes replicate 1 

`ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz` (79.3 Mb)

SRR941819: fermentation 30 minutes replicate 2 

`ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941819/SRR941819.fastq.gz` (282 Mb)

In [2]:
!wget -P data/ ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941816/SRR941816.fastq.gz
!wget -P data/ ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz

!wget -P data/ ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941818/SRR941818.fastq.gz
!wget -P data/ ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941819/SRR941819.fastq.gz

--2021-04-02 22:20:25--  http://ftp.sra.ebi.ac.uk/vol1/fastq/SRR941/SRR941817/SRR941817.fastq.gz
Распознаётся ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)… 193.62.193.138
Подключение к ftp.sra.ebi.ac.uk (ftp.sra.ebi.ac.uk)|193.62.193.138|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 477099436 (455M) [application/octet-stream]
Сохранение в: «data/SRR941817.fastq.gz»

SRR941817.fastq.gz  100%[===================>] 455,00M   163KB/s    за 37m 1s  

2021-04-02 22:57:27 (210 KB/s) - «data/SRR941817.fastq.gz» сохранён [477099436/477099436]



As a reference genome we will use Saccharomyces cerevisiae, we download reference genome and annotation also:

`http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz`

`http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz`

# 2. Aligning with HISAT2

Next, we make genome index and align with hisat2

In [22]:
!conda run hisat2-build data/GCF_000146045.2_R64_genomic.fna data/index

Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:00
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:00
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 12 samples
  (Using difference cover)
  Multikey QSorting samples time: 00:00:00
Calculating bucket sizes
Splitting and merging
  Splitting and merging time: 00:00:00
Avg bucket size: 1.73673e+06 (target: 2279456)
Getting block 1 of 7
  Reserving size (2279457) for bucket 1
  Calculating Z arrays for bucket 1
  Entering block accumulator loop for bucket 1:
  bucket 1: 10%
  bucket 1: 20%
  bucket 1: 30%
  bucket 1: 40%
  bucket 1: 50%
  bucket 1:

Now, run hisat2 in single-end mode

```
conda run hisat2 -p 4 -x data/index -q data/SRR941816.fastq | samtools sort > data/SRR941816.bam
conda run hisat2 -p 4 -x data/index -q data/SRR941817.fastq | samtools sort > data/SRR941817.bam
conda run hisat2 -p 4 -x data/index -q data/SRR941818.fastq | samtools sort > data/SRR941818.bam
conda run hisat2 -p 4 -x data/index -q data/SRR941819.fastq | samtools sort > data/SRR941819.bam
```

Output:

```
9043877 reads; of these:
  9043877 (100.00%) were unpaired; of these:
    512971 (5.67%) aligned 0 times
    7930597 (87.69%) aligned exactly 1 time
    600309 (6.64%) aligned >1 times
94.33% overall alignment rate

9929568 reads; of these:
  9929568 (100.00%) were unpaired; of these:
    505208 (5.09%) aligned 0 times
    8645398 (87.07%) aligned exactly 1 time
    778962 (7.84%) aligned >1 times
94.91% overall alignment rate

1721675 reads; of these:
  1721675 (100.00%) were unpaired; of these:
    65074 (3.78%) aligned 0 times
    1508006 (87.59%) aligned exactly 1 time
    148595 (8.63%) aligned >1 times
96.22% overall alignment rate

6172452 reads; of these:
  6172452 (100.00%) were unpaired; of these:
    229681 (3.72%) aligned 0 times
    5368136 (86.97%) aligned exactly 1 time
    574635 (9.31%) aligned >1 times
96.28% overall alignment rate
```

# Quantifying with featureCounts

Convert GFF file to GTF file

In [5]:
!conda run gffread data/GCF_000146045.2_R64_genomic.gff -T -o data/GCF_000146045.2_R64_genomic.gft

Run the feature counts program:

In [31]:
!conda run featureCounts -g gene_id -a data/GCF_000146045.2_R64_genomic.gft -o data/featureCounts_res.txt data/SRR941816.bam data/SRR941817.bam data/SRR941818.bam data/SRR941819.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.1

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 4 BAM files                                      ||
||                           o SRR941816.bam                                  ||
||                           o SRR941817.bam                                  ||
||                           o SRR941818.bam                                  ||
||                           o SRR941819.bam                     

We don’t need all columns from featureCounts output file for further analysis, so let’s simplify it

In [34]:
!cat data/featureCounts_res.txt | cut -f 1,7-10 > data/simple_counts.txt

# Find differentially expressed genes with Deseq2

calculate metrics:

`cat simple_counts.txt | R -f deseq2.r`

as a result we have 2 files `result.txt` and `norm-matrix-deseq2.txt`

Next draw heatmap:

`cat norm-matrix-deseq2.txt | R -f draw-heatmap.r`

Result:

<img src='img/hit.png'>

# Result Interpretation

take the first 50 genes from `result.txt`

`head -n 50 result.txt | cut -f 1 > genes.txt`

# Use gene ontology terms to get a sense of what these genes are doing

Trim genes.txt

In [45]:
import pandas as pd

df = pd.read_csv('genes.txt', header=None)
df[0] = df[0].str.split('-', 1).str[-1]
df.to_csv('genes_trim.txt', index=None, header=None)

Use: http://www.yeastgenome.org/cgi-bin/GO/goSlimMapper.pl

<img src="img/yeastgene.png">

Results:

In [47]:
df = pd.read_csv('mapper_genes_5768_slimTab.txt', sep='\t')
df.head()

,GOID,TERM,NUM_LIST_ANNOTATIONS,LIST_SIZE,CLUSTER_FREQUENCY,TOTAL_NUM_ANNOTATIONS,POPULATION_SIZE,GENOME_FREQUENCY,ANNOTATED_GENES
0,GO:0006364,rRNA processing,13,49,26.53%,366,6443,5.68%,"YDR449C, YEL026W, YER127W, YGR159C, YHR066W, Y..."
1,GO:0042273,ribosomal large subunit biogenesis,9,49,18.37%,130,6443,2.02%,"YCR072C, YDL063C, YEL026W, YHR066W, YIR012W, Y..."
2,GO:0042274,ribosomal small subunit biogenesis,8,49,16.33%,146,6443,2.27%,"YDR449C, YEL026W, YER127W, YGR159C, YHR196W, Y..."
3,GO:0042255,ribosome assembly,7,49,14.29%,79,6443,1.23%,"YCR072C, YGR159C, YHR066W, YIR012W, YLR264W, Y..."
4,GO:0006811,ion transport,6,49,12.24%,340,6443,5.28%,"YDR536W, YHR094C, YKL120W, YNL065W, YNR060W, Y..."


In [71]:
dfgenes = pd.read_csv('genes_50.txt', sep='\t')[['id', 'log2FoldChange', 'lfcSE', 'padj']]
dfgenes.id = dfgenes.id.str.split('-', 1).str[-1]
dfgenes['TERM'] = ''
dfgenes.head()

,id,log2FoldChange,lfcSE,padj,TERM
0,YER062C,7.899389,0.407320,5.287914e-80,
1,YDR536W,7.873736,0.410492,1.606758e-78,
2,YHR094C,7.881149,0.431514,3.231988e-71,
3,YNL065W,7.784779,0.466557,2.522291e-59,
4,YKL120W,7.268735,0.482387,3.168247e-48,


In [72]:
for i, r in df.iterrows():
    for g in r.ANNOTATED_GENES.split(', '):
        dfgenes.loc[dfgenes.id == g, 'TERM'] = dfgenes.loc[dfgenes.id == g, 'TERM'] + r.TERM + ', '

In [74]:
from IPython.display import display, HTML

display(HTML(dfgenes[dfgenes['log2FoldChange'] > 0].to_html()))

,id,log2FoldChange,lfcSE,padj,TERM
0,YER062C,7.899389,0.407320,5.287914e-80,"carbohydrate metabolic process, response to osmotic stress,"
1,YDR536W,7.873736,0.410492,1.606758e-78,"ion transport, transmembrane transport, carbohydrate transport,"
2,YHR094C,7.881149,0.431514,3.231988e-71,"ion transport, transmembrane transport, carbohydrate transport,"
3,YNL065W,7.784779,0.466557,2.522291e-59,"ion transport, transmembrane transport, response to chemical, amino acid transport,"
4,YKL120W,7.268735,0.482387,3.168247e-48,"ion transport, transmembrane transport,"
5,YJL122W,6.494664,0.431583,3.561524e-48,"ribosomal large subunit biogenesis,"
6,YLR264W,6.483393,0.431127,3.561874e-48,"rRNA processing, ribosomal small subunit biogenesis, ribosome assembly, regulation of translation, RNA catabolic process, nucleobase-containing compound transport, cytoplasmic translation,"
7,YGR159C,6.766882,0.451881,8.098365e-48,"rRNA processing, ribosomal small subunit biogenesis, ribosome assembly, DNA recombination,"
8,YNL112W,7.363432,0.494407,2.442625e-47,"rRNA processing, regulation of translation, RNA catabolic process, DNA-templated transcription, termination, transcription by RNA polymerase II,"
9,YOR271C,6.843726,0.472250,8.264472e-45,"ion transport, transmembrane transport, amino acid transport,"


In [75]:
display(HTML(dfgenes[dfgenes['log2FoldChange'] < 0].to_html()))

,id,log2FoldChange,lfcSE,padj,TERM
27,YLR327C,-5.025613,0.391274,2.001321e-35,
46,YKR097W,-4.708130,0.391535,3.386883e-31,"carbohydrate metabolic process,"
